In [43]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from dotenv import load_dotenv
import asyncio
import os
load_dotenv()

True

In [44]:
api_key=os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Please set the api key in evironemnt Varibale")
openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    api_key=api_key,
)

In [45]:
code_reviewer = AssistantAgent(
    name="CodeReviewer",
    model_client=openai_model_client,
    system_message=(
        "You're a senior software engineer. Your job is to review code for bugs, readability, logic issues, and efficiency. "
        "You give concise feedback, and you also suggest better patterns or simplifications when needed. "
        "Be honest, but encouraging — this is a peer code review. if Completed Please Terminate"
    )
)

dsa_resolver = AssistantAgent(
    name="DSAResolver",
    model_client=openai_model_client,
    system_message=(
        "You're a DSA problem-solving expert. You explain algorithms step by step and write clean Python code for problems. "
        "Always think out loud before coding. Help debug DSA code, analyze time/space complexity, and suggest edge cases. "
        "If needed, break a problem into patterns like sliding window, backtracking, binary search, etc."
    )
)


code_editor = AssistantAgent(
    name="CodeEditor",
    model_client=openai_model_client,
    system_message=(
        "You're an expert coding assistant. Your role is to write, refactor, and extend code with minimal explanation. "
        "Always return only valid and complete code blocks. When modifying existing code, preserve formatting and structure. "
        "Stick to clean, idiomatic code."
    )
)

In [46]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination
my_termination=TextMentionTermination(text='TERMINATION')
team=RoundRobinGroupChat(participants=[dsa_resolver,code_reviewer,code_editor],max_turns=14,termination_condition=my_termination)

In [49]:
async def main():
    task = TextMessage(
        content="write a simple 2 line code in python to add two numbers",
        source='user'
    )
    result=await team.run(task=task)
    for message in result:
        print(f"Source: {message.source}")
        print(f"Message: {message.content}")
        print("-" * 40)


In [50]:
await main()

AttributeError: 'tuple' object has no attribute 'source'

In [ ]:
messages=[TextMessage(id='ff89b290-4002-4f30-8119-170dfd221c3a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 52, 32, 498704, tzinfo=datetime.timezone.utc), content='write a simple 2 line code in python to add two numbers', type='TextMessage'),
          TextMessage(id='e3287923-02ba-46db-a3ca-8fec9eb095d6', source='DSAResolver', models_usage=RequestUsage(prompt_tokens=91, completion_tokens=132), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 52, 35, 544003, tzinfo=datetime.timezone.utc), content='To add two numbers in Python using a simple two-line code, you can do the following:\n\n```python\na, b = 5, 3  # You can change these values to any numbers you want to add\nresult = a + b\nprint(result)  # This will output: 8 for the values 5 and 3\n```\n\nIn this snippet:\n- We first initialize two variables `a` and `b` with the numbers you want to add.\n- We calculate the sum by using the `+` operator and store it in the variable `result`.\n- Finally, we print the result to see the output.', type='TextMessage'),
          TextMessage(id='a0a34b16-f939-4e1e-8135-fec6ce394371', source='CodeReviewer', models_usage=RequestUsage(prompt_tokens=218, completion_tokens=114), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 52, 37, 932552, tzinfo=datetime.timezone.utc), content="Your request can indeed be accomplished in two concise lines. Here's a simplified version:\n\n```python\na, b = 5, 3  # Example values; update them as desired\nprint(a + b)  # Outputs the sum of `a` and `b`\n```\n\nThis accomplishes the task by defining the numbers on the first line and immediately printing their sum on the second line. This approach keeps the code compact and direct for quick calculations. If you intend to perform further operations with the result, storing it as a variable would be more beneficial.", type='TextMessage'),
          TextMessage(id='251ab00d-eb0b-424f-a950-594f33d01906', source='CodeEditor', models_usage=RequestUsage(prompt_tokens=335, completion_tokens=81), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 52, 39, 492585, tzinfo=datetime.timezone.utc), content="Here's a two-line Python code block that adds two numbers and prints the result:\n\n```python\na, b = 5, 3  # Example values; you can change these\nprint(a + b)  # Outputs: 8\n```\n\nThis code initializes two variables with example values and prints their sum directly. Adjust `a` and `b` as needed to add different numbers.", type='TextMessage'),
          TextMessage(id='ef48c740-8628-4d9f-aa4b-0587fe725c4e', source='DSAResolver', models_usage=RequestUsage(prompt_tokens=436, completion_tokens=129), metadata={}, created_at=datetime.datetime(2025, 7, 12, 4, 52, 41, 492774, tzinfo=datetime.timezone.utc), content='You\'ve got it right! This two-line code efficiently initializes and sums two numbers, then prints the result. This is perfect for quick calculations. If you\'re interested in taking input from users, here\'s how you could modify it:\n\n```python\na, b = map(int, input("Enter two numbers separated by space: ").split())\nprint(a + b)\n```\n\nThis version:\n1. Uses `map` and `split` to take input for two numbers from the user in a single line.\n2. Converts the inputs into integers.\n3. Outputs their sum.\n\nThis variant is useful if you want dynamic input rather than hardcoded values.', type='TextMessage')] stop_reason='Maximum number of turns 4 reached.'